In [1]:
#! Install library
import os
import importlib.util

if importlib.util.find_spec("requests") is None:
	os.system("pip install requests")

if importlib.util.find_spec("beautifulsoup4") is None:
	os.system("pip install beautifulsoup4")

if importlib.util.find_spec("Counter") is None:
	os.system("pip install Counter")

In [2]:
#! Authentication details
import os
import dotenv
import importlib.util

JiraHost = None
ConfluenceHost = None
Username = None
Password = None

if importlib.util.find_spec("google.colab") is not None: ## if using google colab
    if not os.path.exists('.env'):
        from google.colab import files
        uploaded = files.upload()
        file_name = list(uploaded.keys())[0]
        try:
            os.rename(file_name, '.env')
        except:
            pass

try:
    dotenv.load_dotenv('../.env', override=True)

    JiraHost = os.getenv('SECRETS_HOST')
    ConfluenceHost = os.getenv('SECRETS_CONFLUENCE')
    Username = os.getenv('SECRETS_USERNAME')
    Password = os.getenv('SECRETS_PASSWORD')
except:
    display("trouble loading dot env")
    pass

if JiraHost is None or JiraHost == "":
    JiraHost = input("Enter Jira Host")

if ConfluenceHost is None or ConfluenceHost == "":
    ConfluenceHost = input("Enter Confluence Host")

if Username is None or Username == "":
    Username = input("Enter Username")

if Password is None or Password == "":
    Password = input("Enter Password")

display("Jira Host: " + JiraHost)
display("Confluence Host: " + ConfluenceHost)

'Jira Host: https://autoandgeneral-sandbox-377.atlassian.net/'

'Confluence Host: https://autoandgeneral-sandbox-377.atlassian.net/wiki/'

In [3]:
#! Functions
import base64
import re
from bs4 import BeautifulSoup
import requests
from collections import Counter
import pandas as pd
import requests
from functools import reduce
import warnings

requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

def _ExpandColumn(self:pd.DataFrame, colName:str, columnsToExpand = [], prefix:str = "Prefix", sentenceCase:bool = True) -> pd.DataFrame:
    if (prefix == "Prefix"):
        prefix = colName + " "
        with warnings.catch_warnings():
          warnings.simplefilter(action='ignore', category=FutureWarning)
          expandedCols = self[colName].apply(lambda x: pd.Series(x).add_prefix(prefix))
        columnsToExpand = [prefix + c for c in columnsToExpand]
    else:
        expandedCols = self[colName].apply(lambda x: pd.Series(x))

    if len(columnsToExpand) > 0:
        expandedCols = expandedCols[columnsToExpand]

    if sentenceCase:
        expandedCols.columns = [fnSentenceCase(c) for c in expandedCols.columns]

    return pd.concat([self.drop(colName, axis=1), expandedCols], axis=1)

pd.DataFrame.expand = _ExpandColumn

def fnSentenceCase(s):
    s = (' '.join(dict.fromkeys(s.split())))  # remove duplicate words
    s = s.replace("0", "") # remove "0"
    s = s.strip()
    return ' '.join([x.capitalize() for x in re.sub(r"([A-Z])", r" \1", s).split()]) # sentence case

def _SentenceCaseColumns(self:pd.DataFrame) -> pd.DataFrame:
    self.columns = [fnSentenceCase(c) for c in self.columns]
    return self

pd.DataFrame.sentence_case_columns = _SentenceCaseColumns

def fnGetDefaultHeaders():
    return {
        "content-type": "application/json",
        "authorization": "Basic " + base64.b64encode((Username + ":" + Password).encode()).decode(),
        "retry-after": "120"
    }

def fnAPI(webRequestDelegate, startAt = 0) -> pd.DataFrame:
    def flatten_reduce_lambda(frm):
        try:
            return list(reduce(lambda x, y: x + y, frm, []))
        except:
            return list(reduce(lambda x, y: x + y, [frm], []))
    def innerGetResults(webRequestDelegate, startAt = 0):
        results = webRequestDelegate(startAt)
        if isinstance(results, dict) and "total" in results and "maxResults" in results:
            if startAt + results["maxResults"] < results["total"]:
                return [results] + innerGetResults(webRequestDelegate, startAt + results["maxResults"])
            else:
                return [results]
        else:
            return [results]
    Source = flatten_reduce_lambda(innerGetResults(webRequestDelegate, startAt))
    df = pd.DataFrame(Source)
    return df

def most_frequent(List):
	try:
		c = Counter(List)
		most_common = [key for key, _ in c.most_common(5)]
		return most_common
	except:
		return ""

In [18]:
import pandas as pd
import json


def fnUpdatePermission(schemeId, permission):
    def ApiCall(schemeId, permission):
        url = "/rest/api/latest/permissionscheme/" + str(schemeId) + "/permission"
        headers = fnGetDefaultHeaders()
        response = requests.post(
            JiraHost + url,
            headers=headers,
            data=json.dumps(permission),
            verify=False,
            allow_redirects=True,
        )
        return response.json()

    df = ApiCall(schemeId, permission)
    return df


def fnGetPermissionSchemes() -> pd.DataFrame:
    def ApiCall(startAt):
        url = "/rest/api/latest/permissionscheme"
        headers = fnGetDefaultHeaders()
        response = requests.get(
            JiraHost + url,
            headers=headers,
            verify=False,
            allow_redirects=True,
        )
        return response.json()

    df = fnAPI(ApiCall)
    df = df.explode("permissionSchemes")
    df = df.expand("permissionSchemes")
    return df


df = fnGetPermissionSchemes()
##df = df[df["Permission Schemes Id"] == 10001]

update = 0
alreadyExists = 0

for index, row in df.iterrows():
    scheme = row
    # New permissions
    p = {}
    p["holder"] = {}
    p["holder"]["type"] = "projectRole"
    p["holder"]["value"] = "10003"
    p["permission"] = "VIEW_AGGREGATED_DATA"

    try:
        response = fnUpdatePermission(scheme["Permission Schemes Id"], p)
        if "errorMessages" in response:
            if not "already exists" in response["errorMessages"][0]:
                raise Exception(response["errorMessages"])
            else:
                alreadyExists = alreadyExists + 1
                pass
        update = update + 1
    except Exception as ex:
        print(ex, scheme["Permission Schemes Name"])
        pass

    p = {}
    p["holder"] = {}
    p["holder"]["type"] = "applicationRole"
    p["permission"] = "VIEW_AGGREGATED_DATA"

    try:
        response = fnUpdatePermission(scheme["Permission Schemes Id"], p)
        if "errorMessages" in response:
            if not "already exists" in response["errorMessages"][0]:
                raise Exception(response["errorMessages"])
            else:
                alreadyExists = alreadyExists + 1
                pass
        update = update + 1
    except Exception as ex:
        print(ex, scheme["Permission Schemes Name"])
        pass

print("Updated = ", update / 2)
print("Already Exists = ", alreadyExists / 2)
print("Total = ", len(df))

['Internal server error'] DYL: Simplified Permission Scheme
['Internal server error'] DYL: Simplified Permission Scheme
['Internal server error'] KPTP: Simplified Permission Scheme
['Internal server error'] KPTP: Simplified Permission Scheme
['Internal server error'] SSPA: Simplified Permission Scheme
['Internal server error'] SSPA: Simplified Permission Scheme
Updated =  41.0
Already Exists =  41.0
Total =  44
